In [19]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import confusion_matrix
from data_loader import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import model
torch.set_printoptions(linewidth=120)
from utils import *
from collections import OrderedDict
from run_builder import RunBuilder
from run_manager import RunManager
from torch.utils.tensorboard import SummaryWriter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000, 10000]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    comment = f'-{run}'
    
    data_loader = DataLoader('fashion_mnist',batch_size=run.batch_size)
    network = model.Network()
    optimizer = optim.Adam(
        network.parameters(), lr=lr
    )
    
    m.begin_run(run, network, data_loader)

    for epoch in range(3):
        m.begin_epoch()
        
        for batch in data_loader.train_loader:
            images, labels = batch # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save('results')
        



,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size
0,1,1,0.984127,0.627517,8.722305,9.466664,0.010,1000
1,1,2,0.551603,0.787100,8.283689,17.864892,0.010,1000
2,1,3,0.459948,0.828900,8.487593,26.462988,0.010,1000
3,2,1,2.119423,0.230717,10.699848,15.647500,0.010,10000
4,2,2,1.517135,0.427200,8.491929,24.251639,0.010,10000
5,2,3,1.111150,0.576283,8.670603,33.032261,0.010,10000
6,3,1,0.989671,0.615183,8.295254,9.004988,0.001,1000
7,3,2,0.546853,0.787500,8.163264,17.282654,0.001,1000
8,3,3,0.459614,0.829183,8.053634,25.448533,0.001,1000
9,4,1,2.194224,0.167200,10.967549,15.742314,0.001,10000
